In [12]:
from xml.dom import minidom
import xml.etree.ElementTree as ET
import re

#Abrimos o ficheiro
doc = minidom.parse("./misinfo-2021-topics.xml")

#Creeamos a lista de pares descripcion,stance
pares = dict()

#Cargamos os nodos de cada tipo
nodes_descriptions = doc.getElementsByTagName('description')
nodes_stnaces = doc.getElementsByTagName('stance')

#Creamos as listas onde almacenamos os elementos
descriptions = []
stances = []

#Percorremos os nodos extraendo os datos
for description in nodes_descriptions:
    descriptions.append(description.firstChild.data)
for stance in nodes_stnaces:
    stances.append(stance.firstChild.data)

#Creamos os pores e introducimolos na lista
for i in range(len(descriptions)):
    if stances[i] == "helpful":
        stances[i] = 'True'
    else:
        stances[i] = 'False'
    pares[descriptions[i]] = (descriptions[i], stances[i])
    
count = 0
for par in pares.values():
    print(str(par) + str(count))
    count = count +1

('Will wearing an ankle brace help heal achilles tendonitis?', 'False')0
('Is a tepid sponge bath a good way to reduce fever in children?', 'False')1
('Can folic acid help improve cognition and treat dementia?', 'False')2
('Does duct tape work for wart removal?', 'False')3
('Should I apply ice to a burn?', 'False')4
('Can vitamin b12 and sun exposure together help treat vitiligo?', 'True')5
('Does yoga improve the management of asthma?', 'True')6
('Is starving a fever effective?', 'False')7
('Does selenium help prevent cancer?', 'False')8
('Will taking birth control pills treat an ovarian cyst?', 'False')9
('Will taking zinc supplements improve pregnancy? ', 'False')10
('Can evening primrose oil help treat eczema?', 'False')11
('Do ankle braces help heal an ankle fracture? ', 'True')12
('Can applying vitamin E cream remove skin scars?', 'False')13
('Can magnesium help prevent migraine?', 'True')14
('Does exercise help manage diabetes?', 'True')15
('Can fermented milk help mitigate high

## Casos de preguntas

In [13]:
variantes = set()
lista = pares.values()

In [14]:
questions = pares

In [15]:
for element in lista:
    variantes.add(element[0].split(" ")[0])

In [16]:
variantes

{'Can', 'Do', 'Does', 'Is', 'Should', 'Will'}

## Creamos a función de conversión

In [17]:
verbs = ['help', 'improve', 'relieve', 'treat', 'remove', 'reduce',
         'work', 'can', 'prevent', 'manage', 'apply', 'dissolve']

In [39]:
'''
This function generates the hand-crafted expression from the query using the answer field 
'''
def generate_aligment(topic):
    description = questions[str(topic)][0] 
    description = re.sub('[^A-Za-z0-9-\s]+', '', description)
    l = description.split()
    
    aux = l[0]
    #l.remove(l[0])
    #l.insert(-2, aux)
    
    if l[0] == "Can" or l[0] == "Will" or l[0] == "Should":
        l.remove(l[0])
        for word in l:
            if word in verbs:
                l.insert(l.index(word), aux.lower())
                break
        result = ' '.join(l)
    elif l[0] == "Do":
        l.remove(l[0])
        result = ' '.join(l)
    elif l[0] == "Does":
        l.remove(l[0])
        for word in l:
            if word in verbs:
                pos = l.index(word)
                l[pos] = l[pos] + 's'
                break
        result = ' '.join(l)
    elif l[0] == "Is":
        l.remove(l[0])
        for i in range(len(l)):
            if l[i] == 'a':
                if i != 0:
                    l.insert(i, aux.lower())
                    break
        result = ' '.join(l)    
    
    result = ' '.join(l)
    return result

In [40]:
for desc in descriptions:
    if desc.split()[0] == "Is":
        print(desc)
        print("Res: " + generate_aligment(desc))

Is a tepid sponge bath a good way to reduce fever in children?
Res: a tepid sponge bath is a good way to reduce fever in children
Is starving a fever effective?
Res: starving is a fever effective
Is the Hoxsey treatment a good cure for cancer?
Res: the Hoxsey treatment is a good cure for cancer


In [41]:
generate_aligment(descriptions[1])

'a tepid sponge bath is a good way to reduce fever in children'

In [42]:
aux = ['a', 'b', 'a']
len(aux)

3

In [43]:
for desc in descriptions:
    print(desc)
    print("Res: " + generate_aligment(desc))

Will wearing an ankle brace help heal achilles tendonitis?
Res: wearing an ankle brace will help heal achilles tendonitis
Is a tepid sponge bath a good way to reduce fever in children?
Res: a tepid sponge bath is a good way to reduce fever in children
Can folic acid help improve cognition and treat dementia?
Res: folic acid can help improve cognition and treat dementia
Does duct tape work for wart removal?
Res: duct tape works for wart removal
Should I apply ice to a burn?
Res: I should apply ice to a burn
Can vitamin b12 and sun exposure together help treat vitiligo?
Res: vitamin b12 and sun exposure together can help treat vitiligo
Does yoga improve the management of asthma?
Res: yoga improves the management of asthma
Is starving a fever effective?
Res: starving is a fever effective
Does selenium help prevent cancer?
Res: selenium helps prevent cancer
Will taking birth control pills treat an ovarian cyst?
Res: taking birth control pills will treat an ovarian cyst
Will taking zinc sup

## Creamos o csv

In [44]:
datos = []
for par in pares.values():
    if par[1] == "True":
        datos.append( '1,"' + generate_aligment(par[0]) + '"')
    if par[1] == "False":
        datos.append( '0,"' + generate_aligment(par[0]) + '"')
#print(datos)#

In [45]:
filename = 'missinfo-claims.csv'
with open(filename, 'w') as f:
    f.write("veracity,claims\n")
    for dato in datos:
        f.write(dato+"\n")

In [46]:
f.close()